# Type Abstraction
An exploration of various techniques and language extensions pertaining to types
 - <a href="#rankntypes">RankNTypes</a> (function parameter type abstraction)
 - <a href="#exquant">Existential Quantification</a> (creating privately scoped fields on a type)
 - <a href="#phantom">Phantom Types</a> (type parameters that don't appear in constructors but encode some meta-data)
 - <a href="#gadts">Generalized Algebraic Datatypes (GADTs)</a> (smart constructors that can be pattern matched)
 - <a href="#typecase">Type Case Pattern</a> (closed type-indexed functions)
 - <a href="#dyn">Dynamic Types</a> (packaging different types under one roof)
 - <a href="#heter">Heterogenous Lists</a> (lists containing more than one type)
 - <a href="#multi">Multiparameter Type-classes and Functional Dependencies</a> (describing relationships between types)


<a id="rankntypes"></a>
## RankNTypes
This is a language extension that allows us to write functions whoes parameters embed polymorphism.


In [30]:
tupleF fn (a, b) = (fn a, fn b)

:t tupleF

tupleF :: forall t b. (t -> b) -> (t, t) -> (b, b)

The compiler has decided that the members of this function's tuple argument are of the same type but this not necessarily what we want:

In [31]:
tupleF show (12, True)

<interactive>:1:14: error:
    • No instance for (Num Bool) arising from the literal ‘12’
    • In the expression: 12
      In the second argument of ‘tupleF’, namely ‘(12, True)’
      In the expression: tupleF show (12, True)

We are trying to pass a polymorphic function to `tupleF` but this polymorphism is not being extended to the type of the tuple argument.

In this case we want Rank2Types

In [32]:
:ext Rank2Types

tupleF' :: (Show a1, Show a2) => (forall a. Show a => a -> b) -> (a1, a2) -> (b, b)
tupleF' fn (a, b) = (fn a, fn b)

tupleF' show (12, True)

("12","True")

ChurchLists are a way to represent a list such that we are only encoding the information necessary to generate that list via a `foldr`. This require the `Rank2Types` extension so that the implementation isn't limited to a `[]`.

In [33]:
-- | Laws:
--
-- > runList xs cons nil == xs
-- > runList (fromList xs) f z == foldr f z xs
-- > foldr f z (toList xs) == runList xs f z
newtype ChurchList a = 
    ChurchList { runList :: forall r. (a -> r -> r) -> r -> r }
 
-- | Make a 'ChurchList' out of a regular list.
fromList :: [a] -> ChurchList a
fromList xs = ChurchList $ \k z -> foldr k z xs
 
-- | Turn a 'ChurchList' into a regular list.
toList :: ChurchList a -> [a]
toList xs = runList xs (:) []
 
-- | The 'ChurchList' counterpart to '(:)'.  Unlike 'DList', whose
-- implementation uses the regular list type, 'ChurchList' abstracts
-- over it as well.
cons :: a -> ChurchList a -> ChurchList a
cons x xs = ChurchList $ \k z -> k x (runList xs k z)
 
-- | Append two 'ChurchList's.  This runs in O(1) time.  Note that
-- there is no need to materialize the lists as @[a]@.
append :: ChurchList a -> ChurchList a -> ChurchList a
append xs ys = ChurchList $ \k z -> runList xs k (runList ys k z)
 
-- i.e.,
 
nil = {- fromList [] = ChurchList $ \k z -> foldr k z []
                  = -} ChurchList $ \k z -> z
 
singleton x = {- cons x nil = ChurchList $ \k z -> k x (runList nil k z) 
            = -} ChurchList $ \k z -> k x z
 
snoc xs x = {- append xs $ singleton x
          = ChurchList $ \k z -> runList xs k (runList (singleton x) k z) 
          = -} ChurchList $ \k z -> runList xs k (k x z)
          
let a = fromList [1,2,3,4]
let b = fromList [5,6,7,8]
let c = append a b
toList c

[1,2,3,4,5,6,7,8]

<a id="exquant"></a>
## Existential Quantification
By default, all parametrically polymorphic functions and types use Universal Quantification. We can demonstrate the difference between Universal and Existential quantification with the following ADT which resembles an OOP object:

In [34]:
data ObjU a = 
       ObjU a        -- object property
       (a -> Bool)   -- methods
       (a -> String)
       
objUF1 (ObjU a f _) = f a
objUF2 (ObjU a _ f) = f a
objUExtract (ObjU a _ _) = a

let obj = ObjU 5 (>2) show
objUF1 obj
objUF2 obj
objUExtract obj

True

"5"

5

But what we want is to encapsulate the property so that only the object's own methods can access it i.e. we shouldn't be able to simply extract it with pattern matching. For this we need Existential Quantification.

In [35]:
:ext ExistentialQuantification

data ObjE = forall a. ObjE a (a -> Bool) (a -> String)

objEF1 (ObjE a f _) = f a
objEF2 (ObjE a _ f) = f a

let obj = ObjE 5 (>2) show
objEF1 obj
objEF2 obj

-- this is illegal, cannot infer type
objEExtract (ObjE a _ _) = a

True

"5"

<interactive>:1:28: error:
    • Couldn't match expected type ‘p’ with actual type ‘a’
        because type variable ‘a’ would escape its scope
      This (rigid, skolem) type variable is bound by a pattern with constructor: ObjE :: forall a. a -> (a -> Bool) -> (a -> String) -> ObjE, in an equation for ‘objEExtract’ at <interactive>:1:14-23
    • In the expression: a
      In an equation for ‘objEExtract’: objEExtract (ObjE a _ _) = a
    • Relevant bindings include
        a :: a (bound at <interactive>:1:19)
        objEExtract :: ObjE -> p (bound at <interactive>:1:1)

This lets us implement abstract datatypes by providing functions over a type while hiding the implementation of that type.

<table style="border-collapse: collapse;border-top: 0.5pt solid ; border-bottom: 0.5pt solid ; " width=""><colgroup><col align="left"><col align="left"></colgroup><thead><tr><th style="border-bottom: 0.5pt solid ; " valign="bottom" align="left">
<p>Universal</p>
</th><th style="border-bottom: 0.5pt solid ; " valign="bottom" align="left">
<p>Existential</p>
</th></tr></thead><tbody><tr><td style="" valign="top" align="left">
<p>Type parametrization</p>
</td><td style="" valign="top" align="left">
<p>Type abstraction</p>
</td></tr><tr><td style="" valign="top" align="left">
<p>Parametric Polymorphism</p>
</td><td style="" valign="top" align="left">
<p>Encapsulation</p>
</td></tr><tr><td style="" valign="top" align="left">
<p>user of data specifies type</p>
</td><td style="" valign="top" align="left">
<p>implementer of data specifies type</p>
</td></tr><tr><td style="" valign="top" align="left">
<p>forall = "for all"</p>
</td><td style="" valign="top" align="left">
<p>forall = "for some"</p>
</td></tr></tbody></table>

<a id="phantom"></a>
## Phantom Types
We have a simple expression language and evaluator:

In [36]:
data Expr1 = I1 Int | Add1 Expr1 Expr1
          
eval1 :: Expr1 -> Int
eval1 (I1 v) = v
eval1 (Add1 a b) = (eval1 a) + (eval1 b)

If we add a second value type, we can no longer define a valid evaluator.

In [37]:
data Expr2 = I2 Int | B2 Bool | Add2 Expr2 Expr2
          
eval2 :: Expr2 -> t
eval2 (B2 v) = v
eval2 (I2 v) = v
eval2 (Add2 a b) = (eval2 a) + (eval2 b)

<interactive>:3:16: error:
    • Couldn't match expected type ‘t’ with actual type ‘Bool’
      ‘t’ is a rigid type variable bound by
        the type signature for:
          eval2 :: forall t. Expr2 -> t
        at <interactive>:2:1-19
    • In the expression: v
      In an equation for ‘eval2’: eval2 (B2 v) = v
    • Relevant bindings include eval2 :: Expr2 -> t (bound at <interactive>:3:1)

And we can also construct bad types

In [38]:
let bad = Add2 (I2 3) (B2 False)

<interactive>:1:11: error:
    • Data constructor not in scope: Add2 :: t0 -> t1 -> t
    • Perhaps you meant one of these: ‘Ghci88.Add1’ (imported from Ghci88), ‘Add1’ (line 1), ‘Add3’ (line 1)
<interactive>:1:17: error:
    • Data constructor not in scope: I2 :: Integer -> t0
    • Perhaps you meant one of these: ‘Ghci88.I1’ (imported from Ghci88), ‘I1’ (line 1), ‘I3’ (line 1)
<interactive>:1:24: error:
    • Data constructor not in scope: B2 :: Bool -> t1
    • Perhaps you meant one of these: ‘B3’ (line 1), ‘B4’ (line 4)

We can solve the second problem by using a Phantom Type.

In [39]:
data Expr3 t = I3 Int | B3 Bool | Add3 (Expr3 Int) (Expr3 Int)

`t` doesn't appear in any constructors, hence 'phantom'.

We can use the phantom type to write the following 'smart constructors':

In [40]:
i3 x = I3 x :: Expr3 Int
b3 x = B3 x :: Expr3 Bool
add3 a b = Add3 a b :: Expr3 Int

-- this does not compile
let bad = add3 (b3 True) (i3 3)

<interactive>:1:17: error:
    • Couldn't match type ‘Bool’ with ‘Int’
      Expected type: Expr3 Int
        Actual type: Expr3 Bool
    • In the first argument of ‘add3’, namely ‘(b3 True)’
      In the expression: add3 (b3 True) (i3 3)
      In an equation for ‘bad’: bad = add3 (b3 True) (i3 3)

However, our type `Expr3 t` is ambiguous, so we still cannot write an evaluator

In [41]:
eval3 :: Expr3 t -> t
eval3 (B3 v) = v
eval3 (I3 v) = v
eval3 (Add3 a b) = eval3 a + eval3 b

<interactive>:2:16: error:
    • Couldn't match expected type ‘t’ with actual type ‘Bool’
      ‘t’ is a rigid type variable bound by
        the type signature for:
          eval3 :: forall t. Expr3 t -> t
        at <interactive>:1:1-21
    • In the expression: v
      In an equation for ‘eval3’: eval3 (B3 v) = v
    • Relevant bindings include eval3 :: Expr3 t -> t (bound at <interactive>:2:1)

<a id="gadts"></a>
## Generalized Algebraic Datatypes (GADTs)
GADTs bring together phantom types, smart constructors, and refined pattern matching. We can use them to solve our evaluator dilemma.

In [42]:
:ext GADTs

data Expr4 t where -- phantom type
  -- built-in smart constructors
  I4 :: Int -> Expr4 Int
  B4 :: Bool -> Expr4 Bool
  Add4 :: Expr4 Int -> Expr4 Int -> Expr4 Int
  
eval4 :: Expr4 t -> t
eval4 (I4 v) = v -- pattern matching on the constructor is no longer ambiguous
eval4 (B4 v) = v
eval4 (Add4 a b) = eval4 a + eval4 b

eval4 $ Add4 (I4 4) (I4 5)

9

<a id="typecase"></a>
## Type Case Pattern
Let's see another way to leverage GADTs. We'll define a type that unifies `Int`, `Char`, and `[]`.

In [43]:
data Rep t where
  RInt :: Rep Int
  RChar :: Rep Char
  RList :: Show a => Rep a -> Rep [a] -- 'a' is existentially quantified (not on the left side)

We'll write a function that takes a value along with it's type representation.

In [44]:
showT :: Show t => Rep t -> t -> String

showT RInt x = show x ++ " :: INT"
showT RChar x = show x ++ " :: CHAR"
showT (RList _) [] = "The End"
showT (RList rep) (x:xs) = showT rep x ++ ", " ++ showT (RList rep) xs

showT RInt 5
showT RChar '!'
showT (RList RInt) [1,2,3,4]

"5 :: INT"

"'!' :: CHAR"

"1 :: INT, 2 :: INT, 3 :: INT, 4 :: INT, The End"

`showT` is considered a closed type-indexed function because the type index family (`RepT`) is fixed. Contrast this with the standard `show` which is an open type-indexed function because the `Show` type class can be implemented for any types we want.

<a id="dyn"></a>
## Dynamic Types
To make a dynamic type, we will package a type with it's representation. For starters, we'll do the packaging with existential quantification.

In [45]:
data DynamicEQ = forall t. Show t => DynEQ (Rep t) t

Even though `DynEQ` dynamic values have opaque type, they are well typed. For example, we can use them to express heterogeneous lists:

In [46]:
dynList = [DynEQ RInt 5, DynEQ RChar 'a']

GADTs generalize existentials, so we can express the above as a GADT:

In [47]:
data Dynamic where 
  Dyn :: Show t => Rep t -> t -> Dynamic
  
instance Show Dynamic where
  show (Dyn r t) = showT r t
  
dynList = [Dyn RInt 100, Dyn RChar 'b']
map show dynList

["100 :: INT","'b' :: CHAR"]

We can perform type safe casts on dynamic types:

In [48]:
toInt :: Dynamic -> Maybe Int
toInt (Dyn RInt v) = Just v
toInt _ = Nothing

toChar :: Dynamic -> Maybe Char
toChar (Dyn RChar v) = Just v
toChar _ = Nothing

map toInt dynList
map toChar dynList

[Just 100,Nothing]

[Nothing,Just 'b']

<a id="heter"></a>
## Heterogenous Lists
We can define heterogenous lists (lists of varying types) using both existentials and GADTs.

With existentials:

In [49]:
:ext ExistentialQuantification

data LI_Eq1 = forall a. LI_Eq1 a

hListEq1 :: [LI_Eq1]
hListEq1 = [LI_Eq1 5, LI_Eq1 "3"]

Because the type of `a` cannot be infered, we can't do anything with this list. We would need to package a `show` function with the type to be able to print it:

In [50]:
data LI_Eq2 = forall a. LI_Eq2 a (a -> String)

hListEq2 = [LI_Eq2 5 show, LI_Eq2 "3" show]

map (\(LI_Eq2 v f) -> f v) hListEq2

["5","\"3\""]

We could've also used a constraint to avoid having to package the show function (Bounded Quantification):

In [51]:
data LI_Eq3 = forall a. Show a => LI_Eq3 a

hListEq3 = [LI_Eq3 1, LI_Eq3 (Just 9)]
map (\(LI_Eq3 a) -> show a) hListEq3

["1","Just 9"]

We can use the same two strategies to express heterogenous lists using GADTs

In [52]:
data LI_Gadt1 where
  MkShow1 :: a -> (a -> String) -> LI_Gadt1
  
data LI_Gadt2 where
  MkShow2 :: Show a => a -> LI_Gadt2 -- Bounded Quantification

<a id="multi"></a>
## Multiparameter Type-Classes

Regular type classes specify a set of types: `Monad a`, `Ord a` etc. Multiparameter type classes specify a relationship between types by way of functional dependencies. This can be seen with the `Coerce` class:

In [53]:
:ext MultiParamTypeClasses
:ext FlexibleInstances

class Coerce1 a b where
  coerce1 :: a -> b
  
instance Coerce1 Int String where
  coerce1 = show
  
instance Coerce1 Int [Int] where
  coerce1 = (:[])
  
coerce1 (12 :: Int) :: String
coerce1 (12 :: Int) :: [Int]

"12"

[12]

Type inference suffers with multiparameter types, that's why we had to use annotations so liberally.

With the advent of Functional Dependencies, multiparameter type classes became much more practical

In [57]:
:ext FunctionalDependencies

class Coerce2 a b | b -> a where
  coerce2 :: a -> b
  
instance Coerce2 Int String where
  coerce2 = show
  
coerce2 5 :: String

"5"

The `b -> a` tells the compiler that there can only be one type of `a` corresponding to a type `b`. If we try to make another instance using the same `b`, we get an error because `String` is already associated with `Int`:

In [58]:
instance Coerce2 Float String where
  coerce2 = show

<interactive>:1:10: error:
    Functional dependencies conflict between instance declarations:
      instance Coerce2 Float String -- Defined at <interactive>:1:10
      instance [safe] Coerce2 Int String -- Defined at <interactive>:3:10